In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential 
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [2]:
# Function to create a dataframe with image paths and labels
def createDataFrame(dir):
    image_path = []
    labels = []

    for label in os.listdir(dir):
        label_path = os.path.join(dir, label)
        if os.path.isdir(label_path):  
            for imagename in os.listdir(label_path):  
                image_path.append(os.path.join(label_path, imagename))
                labels.append(label)
    return image_path, labels

In [3]:
# Define data directories
train_data_dir = 'D:/python/images/train'  
test_data_dir = 'D:/python/images/test'

In [4]:
# Create training and testing dataframes
train = pd.DataFrame()
train['image'], train['label'] = createDataFrame(train_data_dir)

test = pd.DataFrame()
test['image'], test['label'] = createDataFrame(test_data_dir)

In [5]:
# Function to extract features from images
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode="grayscale", target_size=(48, 48))  # Resize images to 48x48
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)  # Reshape for Conv2D
    return features

In [6]:
# Extract features for training and testing sets
train_features = extract_features(train['image'])
test_features = extract_features(test['image'])



00%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 7066/7066 [00:01<00:00, 4639.42it/s]

In [7]:
# Normalize the features
x_train = train_features / 255.0
x_test = test_features / 255.0

In [8]:
# Encode labels into integers
le = LabelEncoder()
le.fit(train['label'])

y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [9]:
# Define the model architecture
model2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(7, activation='softmax')  # Assuming 7 classes
])


D:\python\myenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Compile the model
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Print the model summary
model2.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 46, 46, 16)          │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 46, 46, 16)          │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 23, 23, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 23, 23, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 21, 21, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 21, 21, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 10, 10, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 10, 10, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 8, 8, 64)            │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 8, 8, 64)            │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 4, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 4, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 2, 2, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 2, 2, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 1, 1, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 1, 1, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 115,527 (451.28 KB)

 Trainable params: 115,047 (449.40 KB)

 Non-trainable params: 480 (1.88 KB)

In [ ]:
# Train the model
history = model2.fit(x_train, y_train, epochs=300, validation_split=0.1, batch_size=128)

# Evaluate the model on the test set
test_loss, test_acc = model2.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc}")

# Save the trained model for future use
model2.save('emotion_recognition_model.h5')
print("Model saved as 'emotion_recognition_model.h5'")

Epoch 1/300
203/203 ━━━━━━━━━━━━━━━━━━━━ 15s 75ms/step - accuracy: 0.7119 - loss: 0.7507 - val_accuracy: 0.2522 - val_loss: 3.1407
Epoch 2/300
203/203 ━━━━━━━━━━━━━━━━━━━━ 15s 76ms/step - accuracy: 0.7199 - loss: 0.7320 - val_accuracy: 0.3358 - val_loss: 2.3467
Epoch 3/300
203/203 ━━━━━━━━━━━━━━━━━━━━ 16s 78ms/step - accuracy: 0.7187 - loss: 0.7435 - val_accuracy: 0.1953 - val_loss: 3.6907
Epoch 4/300
203/203 ━━━━━━━━━━━━━━━━━━━━ 16s 80ms/step - accuracy: 0.7182 - loss: 0.7317 - val_accuracy: 0.2771 - val_loss: 2.8346
Epoch 5/300
203/203 ━━━━━━━━━━━━━━━━━━━━ 17s 83ms/step - accuracy: 0.7196 - loss: 0.7358 - val_accuracy: 0.1602 - val_loss: 4.0082
Epoch 6/300
203/203 ━━━━━━━━━━━━━━━━━━━━ 17s 86ms/step - accuracy: 0.7243 - loss: 0.7316 - val_accuracy: 0.2310 - val_loss: 3.1028
Epoch 7/300
203/203 ━━━━━━━━━━━━━━━━━━━━ 17s 85ms/step - accuracy: 0.7176 - loss: 0.7458 - val_accuracy: 0.1270 - val_loss: 3.8810
Epoch 8/300
203/203 ━━━━━━━━━━━━━━━━━━━━ 17s 85ms/step - accuracy: 0.7294 - loss: 0